### Load libraries and functions

In [1]:
#Load libraries
library(Seurat)
library(stringr)
library(viridis)
library(ggplot2)
library(cowplot)
library(cluster)
library(data.table)
library(foreach)
library(doParallel)
library(proxy)
library(ComplexHeatmap)
library(circlize)
library(igraph)
library(qvalue)
library(dplyr)
library(viridis)
library(VGAM)
library(forcats)
library(grDevices)
library(graphics)
library(RColorBrewer)
library(pheatmap)
library(Cairo)
library(reshape2)
library(R.utils)
set.seed(seed = 42)

Attaching SeuratObject

Loading required package: viridisLite

Loading required package: iterators

Loading required package: parallel


Attaching package: ‘proxy’


The following objects are masked from ‘package:stats’:

    as.dist, dist


The following object is masked from ‘package:base’:

    as.matrix


Loading required package: grid

ComplexHeatmap version 2.11.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))


circlize version 0.4.14
CRA

In [2]:
#load basic functions
createEmptyDf = function( nrow, ncol, colnames = c() ){
  data.frame( matrix( vector(), nrow, ncol, dimnames = list( c(), colnames ) ) )
}

tableread_fast = function(i, header=TRUE, quote="", sep=","){
  tmp = fread(i, header=header, sep=sep, quote=quote, nThread=32)
  tmp = as.data.frame(tmp)
  return(tmp)
}

### Create Seurat object

In [3]:
#load data
#Put gene list in the target panel in the working directory ("BD_genes_plus25")

###Input layer
set.seed(seed = 42)

sample.name <- "Aoki"

################################ Processing layer ##########################################
#Create input file name
input_name <- str_c("matrix_inflection_demulti_", sample.name, ".txt", sep = "")
input_dir <- str_c("raw.data/", sample.name, "_results/matrix/", sep = "")
input_name <- str_c(input_dir, input_name, sep = "") #Name of dataset1

#Unzip input file
compressed.name <- str_c(input_name, ".gz", sep = "")
gunzip(compressed.name)

#Create Seurat object
matrix <- tableread_fast(input_name, header = TRUE, quote="", sep="\t")
row.names(matrix) <- matrix$V1
matrix <- dplyr::select(matrix, -V1)
seu <- CreateSeuratObject(counts=matrix, project = "seu", min.cells = 3, min.features = 10)

In [4]:
#Output cell barcode information for scTCR analysis

###Input layer
out.dir <- "scTCR_rawdata"

############################## Processing layer ########################################

dir.create(out.dir)
name.output <- str_c(sample.name, "BC_target.txt", sep = "_")
name.output <- str_c(out.dir, name.output, sep = "/")

BC <- row.names(seu@meta.data)
BC <- str_split(BC, pattern = "_", simplify = TRUE)
BC <- BC[,3]
write.table(BC, name.output, row.names = FALSE, quote = FALSE, col.names = FALSE)

Warning message in dir.create(out.dir):
“ 'scTCR_rawdata' はすでに存在します ”


### Initial analysis: check non-T cell contamination

### Filtering and dimentional reduction

In [5]:
###Gene filtering

##Input layer
dir.name=("Seurat_plots/")
gene.list.name <- "BD_genes_plus10.txt"

########################## Processing layer ###############################
dir.create(dir.name)

##load gene list of BD target panel
BD_genes <- read.table(gene.list.name, header = TRUE)
BD_genes <- as.vector(BD_genes$Genesymbol)

##Chose BD target gene list for downstream analysis
res1 = seu@assays$RNA@counts
res1 =res1[res1@Dimnames[[1]] %in% BD_genes,]
ngenes <- length(res1@Dimnames[[1]])
seu@assays$RNA@counts <- res1
seu@assays$RNA@data <- res1

#Scatter plot for gene/read count
file.name=paste(dir.name, sample.name, "Reads.Genes.png", sep='')
png(file.name, width = 512, height = 400)
FeatureScatter(object = seu, feature1 = "nCount_RNA", feature2 = "nFeature_RNA", pt.size = 0.3) +
  geom_hline(yintercept=10) +
  scale_x_log10()
dev.off()

#RidgePlot for gene/read count, 
file.name=paste(dir.name, sample.name, "nCount_RNA_log.png", sep='')
png(file.name, width = 512, height = 400)
RidgePlot(object = seu, features = "nCount_RNA", group.by="orig.ident", ncol = 1) +
  scale_x_log10()
dev.off()

file.name=paste(dir.name, sample.name, "nGenes.png", sep='')
png(file.name, width = 512, height = 400)
RidgePlot(object = seu, features = "nFeature_RNA", group.by="orig.ident", ncol = 1)
dev.off()

#Remove doublet, Sampletag-undetected, and Pmel-Sampletag detected cells
seu <- subset(x=seu, idents = c("mouseSampleTag5", "mouseSampleTag6"))

png 
  2

Scale for 'x' is already present. Adding another scale for 'x', which will
replace the existing scale.

Picking joint bandwidth of 0.0642



png 
  2

Picking joint bandwidth of 7.89



png 
  2

In [6]:
##normalizing data/PCA and JackStraw

########################## Processing layer ###############################
# normalizing data
seu = NormalizeData(object = seu, scale.factor=1000000)
VariableFeatures(seu) <- rownames(seu[["RNA"]]) #Use all genes in target panel for Variable features
ngenes <- length(x = seu@assays$RNA@var.features)
all.genes <- rownames(seu)
seu = ScaleData(object = seu, vars.to.regress = c("nCount_RNA"), features = all.genes) 

#Perform PCA
seu = RunPCA(object = seu, features = seu@assays$RNA@var.features, npcs = 100)

#JackStraw
seu = JackStraw(object = seu, num.replicate = 100, dims = 100)
seu <- ScoreJackStraw(object = seu, dims = 1:100, score.thresh = 0.05)
file.name=paste(sample.name, "Jackstraw.png", sep='.')　
file.name=paste(dir.name, file.name, sep='/')　
png(file.name, width = 1250, height = 500)
JackStrawPlot(object = seu, dims = 1:100)
dev.off()

#Determine PCs used for clustering/tSNE analysis (dims.use)
#Extract PCs which fulfill the pvalue threshold
tmp = as.data.frame(seu@reductions$pca@jackstraw@overall.p.values)
tmp1 = tmp[tmp$Score>0.05,1]
dims= c(1:(min(tmp1)-1))

Regressing out nCount_RNA

Centering and scaling data matrix

PC_ 1 
Positive:  Thy1, Nkg7, Trbc2, Trac, Cd8b1, Cd3g, Cd8a, Il2rb, Lat, Pdcd1 
	   Lck, Ctla4, Lag3, Cxcr6, Gzmb, Cd3e, Zap70, Lgals1, Cd3d, Cd2 
	   Havcr2, Cst7, Ctsw, Ifng, Tnfrsf9, Cd247, Tyms, S100a10, Top2a, Cd48 
Negative:  Cd14, Fcgr3, Cxcl2, Il1rn, Ly86, Adgre1, Trem2, Slc11a1, Cxcl16, Cd300a 
	   Ccl2, Tnfsf13, Cd74, Cd36, Cd63, C3, Clec4d, H2-Ab1, H2-Eb1, Nlrp3 
	   H2-Aa, Itgam, Ccr1, Ifi204, Lyz2, Mgst1, Il1b, Thbs1, Ccl6, Apoe 
PC_ 2 
Positive:  Lgals3, Gzmb, Ccl5, Lgals1, Cxcr6, Pdcd1, Ccr2, Ctsd, Gzmk, S100a10 
	   Klrc1, Nkg7, Ctla4, Lag3, Havcr2, Cd8b1, Ccr5, Fasl, Entpd1, Itga4 
	   Ifng, Lyz2, Klrk1, Il1rn, Cd52, Cd14, Cxcl2, Adgre1, Itgax, Ccl6 
Negative:  Tcf7, Ccr7, Sell, Il7r, Cxcr5, Klra6, Klra7, Myc, Ighm, Xcl1 
	   Il4ra, Tnfsf8, Nt5e, Fas, Cd69, Btla, H2-K1, Stat4, Cd9, Tnfrsf25 
	   Ptprc, Klra1, Junb, Stat5a, Bcl6, Itgae, Il6ra, Btg1, Trat1, Pik3ip1 
PC_ 3 
Positive:  Tnfrsf4, Gapdh, Tnfrsf9, 

png 
  2

In [7]:
###Perform tSNE

seu <- RunTSNE(object = seu, dims.use = 1:(min(tmp1)-1), do.fast = TRUE)

### Initial Clustering and check contamination

In [8]:
###Clustering

##Input layer
#Set resolution for clustering
resol <- 1.0

########################## Processing layer ###############################
seu = FindNeighbors(seu, reduction = "pca", dims = dims, force.recalc = TRUE)
seu <- FindClusters(object = seu, resolution =resol)


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 13625
Number of edges: 383756

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7551
Number of communities: 14
Elapsed time: 3 seconds


In [9]:
##Output dimentional reduction with cluster identities

########################### Processing layer ###########################

p1 = DimPlot(object = seu, reduction = "tsne", label = TRUE, label.size = 10, pt.size = 0.5) +
  theme(axis.title.x = element_text(size=10, family = "Arial"), 
        axis.title.y = element_text(size=10, family = "Arial"), 
        axis.text.x = element_text(size=10, colour = 1, family = "Arial"), 
        axis.text.y = element_text(size = 10, colour = 1, family = "Arial")) +
  theme(panel.border = element_rect(fill = NA, size = 1)) 
p2 = DimPlot(object = seu, reduction = "tsne", label = FALSE, label.size = 10, pt.size = 0.5, group.by = "orig.ident") +
  theme(axis.title.x = element_text(size=10, family = "Arial"),
        axis.title.y = element_text(size=10, family = "Arial"),
        axis.text.x = element_text(size=10, colour = 1, family = "Arial"),
        axis.text.y = element_text(size = 10, colour = 1, family = "Arial")) +
  theme(panel.border = element_rect(fill = NA, size = 1)) 

legend1 <- cowplot::get_legend(p1)
legend2 <- cowplot::get_legend(p2)
p1 = p1 + theme(legend.position = 'none')
p2 = p2 + theme(legend.position = 'none')
file.name=paste(sample.name, "tsne_reso", resol, ".png", sep='')
file.name=paste(dir.name, file.name, sep='/')
save_plot(file = file.name, plot_grid(p1, legend1, p2, legend2, ncol=2, nrow=2), device="png", 
          units="in", dpi = 600, base_width = 10, base_height = 10, limitsize=FALSE)

Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message 

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x

In [10]:
##Extract marker genes

###Input layer
dir.name <- "MarkerGenePlot"

########################## Processing layer #############################
seu.markers = FindAllMarkers(seu, verbose = TRUE, test.use="wilcox", only.pos=TRUE, min.pct=0.1, features.use = NULL, return.thresh=0.05)

#Create heatmap with top10 marker genes
dir.name <- "MarkerGenePlot"
dir.create(dir.name)
top10 = seu.markers %>% group_by(cluster) %>% top_n(10, avg_log2FC)
top10 = as.data.frame(top10)
top10  = top10 [!duplicated(top10$gene),]
top10 = top10 %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
top10 = as.data.frame(top10)
file.name=paste(sample.name, "marker_res", resol, ".png", sep='')
file.name <-paste(dir.name, file.name, sep = "/")
p <- DoHeatmap(seu, features = top10$gene, disp.min = -2.5, disp.max = 2.5, size = 8)
ggsave(file = file.name, plot = p, device="png", units="in", dpi = 300,
       width = 20, height = 20, limitsize=FALSE)

seu.markers$cluster = as.numeric(seu.markers$cluster)
seu.markers = seu.markers %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
seu.markers = as.data.frame(seu.markers)
file.name=paste(sample.name, "ALLmarkers_minpct0.1_Adj_p0.05.txt", sep='')
file.name <-paste(dir.name, file.name, sep = "/")
fwrite(seu.markers, file.name, row.names=F, col.names=T, sep="\t", quote=F)
file.name=paste(sample.name, "ALLmarkers_minpct0.1_Adj_p0.05.rda", sep='')
file.name <-paste(dir.name, file.name, sep = "/")
save(seu.markers, file=file.name)

#Output Seurat object in initial analysis
file.name=paste(sample.name, "_res", resol, ".rda", sep='')
save(seu, file=file.name)

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 11

Calculating cluster 12

Calculating cluster 13



In [11]:
#Check marker genes for lineage
tmp_1 = c("Trbc2", "Cd3e", "Cd8a", "Cd4", "Cd14", "Lyz2")

tmps <- list(tmp_1)
tmp_names <- c("lineage")

########################## Processing layer #############################
dir.name="Marker.Scatter"
dir.create(dir.name)

for (i in 1:length(tmp_names)){
  tmp <- tmps[[i]]
  tmp_name <- tmp_names[i]
  
  tmp_name <- paste(tmp_name, "png", sep = ".")
  file.name = paste(dir.name, tmp_name, sep='/')
  png(file.name, width = 1536, height = 240)
  p <- FeaturePlot(seu, features = tmp, ncol = 6, order = TRUE,
                   reduction = "tsne", dims=c(1,2), cols = c("grey", "red"), pt.size = 0.2)
  plot(p)
  dev.off()
}

dir.name="Marker.Violin"
dir.create(dir.name)

for (i in 1:length(tmp_names)){
  tmp <- tmps[[i]]
  tmp_name <- tmp_names[i]
  
  tmp_name <- paste(tmp_name, "png", sep = ".")
  file.name = paste(dir.name, tmp_name, sep='/')
  png(file.name, width = 1536, height = 240)
  p <- VlnPlot(seu, features = tmp, ncol = 6, pt.size = 0.1)
  plot(p)
  dev.off()
}


### 2nd analysis with removing contaminant non-T cells

In [12]:
###Remove contaminant cluster and repeat Seurat pipelines

###Input layer 
sample.name <- "Aoki_2nd"
dir.name <- "Seurat_plot_2nd"
#Resolution for clustering
resol <- 0.6

#Choose subcluster for analyse
seu <- subset(seu, idents = c(0, 1, 2, 3, 4, 5, 6, 7, 8, 9))

########################## Processing layer #############################
dir.create(dir.name)

# normalizing data
seu = NormalizeData(object = seu, scale.factor=1000000)
VariableFeatures(seu) <- rownames(seu[["RNA"]]) #Variable featuresには全検出遺伝子を指定
ngenes <- length(x = seu@assays$RNA@var.features)
all.genes <- rownames(seu)
seu = ScaleData(object = seu, vars.to.regress = c("nCount_RNA"), features = all.genes) #細胞ごとのリード数の補正

# perform PCA
seu = RunPCA(object = seu, features = seu@assays$RNA@var.features, npcs = 100)

#JackStraw
seu = JackStraw(object = seu, num.replicate = 100, dims = 100)
seu <- ScoreJackStraw(object = seu, dims = 1:100, score.thresh = 0.05)
file.name=paste(sample.name, "Jackstraw.png", sep='.')　#PCに濃縮されている遺伝子セットを抽出
file.name=paste(dir.name, file.name, sep='/')　
png(file.name, width = 1250, height = 500)
JackStrawPlot(object = seu, dims = 1:100)
dev.off()

#Determine PCs used for clustering/tSNE analysis (dims.use)
#Extract PCs which fulfill the pvalue threshold
tmp = as.data.frame(seu@reductions$pca@jackstraw@overall.p.values)
tmp1 = tmp[tmp$Score>0.05,1]
dims= c(1:(min(tmp1)-1))

##Perform tSNE
seu <- RunTSNE(object = seu, dims.use = 1:(min(tmp1)-1), do.fast = TRUE)

##Find clusters
seu = FindNeighbors(seu, reduction = "pca", dims = dims, force.recalc = TRUE)
seu <- FindClusters(object = seu, resolution =resol)

In [16]:
##Output dimentional reduction with cluster identities
dir.name <- "Seurat_plot_2nd"

########################## Processing layer #############################
p1 = DimPlot(object = seu, reduction = "tsne", label = TRUE, label.size = 10, pt.size = 0.5) +
  theme(axis.title.x = element_text(size=10, family = "Arial"), 
        axis.title.y = element_text(size=10, family = "Arial"), 
        axis.text.x = element_text(size=10, colour = 1, family = "Arial"), 
        axis.text.y = element_text(size = 10, colour = 1, family = "Arial")) +
  theme(panel.border = element_rect(fill = NA, size = 1)) 
p2 = DimPlot(object = seu, reduction = "tsne", label = FALSE, label.size = 10, pt.size = 0.5, group.by = "orig.ident") +
  theme(axis.title.x = element_text(size=10, family = "Arial"),
        axis.title.y = element_text(size=10, family = "Arial"),
        axis.text.x = element_text(size=10, colour = 1, family = "Arial"),
        axis.text.y = element_text(size = 10, colour = 1, family = "Arial")) +
  theme(panel.border = element_rect(fill = NA, size = 1)) 

legend1 <- cowplot::get_legend(p1)
legend2 <- cowplot::get_legend(p2)
p1 = p1 + theme(legend.position = 'none')
p2 = p2 + theme(legend.position = 'none')
file.name=paste(sample.name, "tsne_reso", resol, ".png", sep='')
file.name=paste(dir.name, file.name, sep='/')
save_plot(file = file.name, plot_grid(p1, legend1, p2, legend2, ncol=2, nrow=2), device="png", 
          units="in", dpi = 600, base_width = 10, base_height = 10, limitsize=FALSE)

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x

In [17]:
##Extract marker genes

###Input layer
dir.name <- "MarkerGenePlot"


############################Processing layer ###########################
dir.create(dir.name)
seu.markers = FindAllMarkers(seu, verbose = TRUE, test.use="wilcox", only.pos=TRUE, min.pct=0.1, features.use = NULL, return.thresh=0.05)

#Create heatmap with top10 marker genes
top10 = seu.markers %>% group_by(cluster) %>% top_n(10, avg_log2FC)
top10 = as.data.frame(top10)
top10  = top10 [!duplicated(top10$gene),]
top10 = top10 %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
top10 = as.data.frame(top10)
file.name=paste(sample.name, "marker_res", resol, ".png", sep='')
file.name <-paste(dir.name, file.name, sep = "/")
p <- DoHeatmap(seu, features = top10$gene, disp.min = -2.5, disp.max = 2.5, size = 8)
ggsave(file = file.name, plot = p, device="png", units="in", dpi = 300,
       width = 20, height = 20, limitsize=FALSE)

seu.markers$cluster = as.numeric(seu.markers$cluster)
seu.markers = seu.markers %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
seu.markers = as.data.frame(seu.markers)
file.name=paste(sample.name, "ALLmarkers_minpct0.1_Adj_p0.05.txt", sep='')
file.name <-paste(dir.name, file.name, sep = "/")
fwrite(seu.markers, file.name, row.names=F, col.names=T, sep="\t", quote=F)
file.name=paste(sample.name, "ALLmarkers_minpct0.1_Adj_p0.05.rda", sep='')
file.name <-paste(dir.name, file.name, sep = "/")
save(seu.markers, file=file.name)

#Output Seurat object in 2nd analysis
file.name=paste(sample.name, "_res", resol, ".rda", sep='')
save(seu, file=file.name)

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Warning message in dir.create(dir.name):
“ 'MarkerGenePlot' はすでに存在します ”


### 3rd analysis with removing contaminant non-T cells

In [18]:
###Remove contaminant cluster and repeat Seurat pipelines

###Input layer 
sample.name <- "Aoki_3rd"
dir.name <- "Seurat_plot_3rd"
#Resolution for clustering
resol <- 0.6

#Choose subcluster for analyse
seu <- subset(seu, idents = c(0, 1, 2, 3, 4, 5, 6))

#####################Processing layer#########################################
dir.create(dir.name)

# normalizing data
seu = NormalizeData(object = seu, scale.factor=1000000)
VariableFeatures(seu) <- rownames(seu[["RNA"]]) #Use all genes in target panel for Variable features
ngenes <- length(x = seu@assays$RNA@var.features)
all.genes <- rownames(seu)
seu = ScaleData(object = seu, vars.to.regress = c("nCount_RNA"), features = all.genes) 

#Perform PCA
seu = RunPCA(object = seu, features = seu@assays$RNA@var.features, npcs = 100)

#JackStraw
seu = JackStraw(object = seu, num.replicate = 100, dims = 100)
seu <- ScoreJackStraw(object = seu, dims = 1:100, score.thresh = 0.05)
file.name=paste(sample.name, "Jackstraw.png", sep='.')　#PCに濃縮されている遺伝子セットを抽出
file.name=paste(dir.name, file.name, sep='/')　
png(file.name, width = 1250, height = 500)
JackStrawPlot(object = seu, dims = 1:100)
dev.off()

#Determine PCs used for clustering/tSNE analysis (dims.use)
#Extract PCs which fulfill the pvalue threshold
tmp = as.data.frame(seu@reductions$pca@jackstraw@overall.p.values)
tmp1 = tmp[tmp$Score>0.05,1]
dims= c(1:(min(tmp1)-1))

##Perform UMAP
seu <- RunUMAP(seu, dims = dims)

##Find clusters
seu = FindNeighbors(seu, reduction = "pca", dims = dims, force.recalc = TRUE)
seu <- FindClusters(object = seu, resolution =resol)

In [33]:
##Output dimentional reduction with cluster identities
dir.name <- "Seurat_plot_3rd"

############################Processing layer ###########################

p1 = DimPlot(object = seu, reduction = "umap", label = TRUE, label.size = 10, pt.size = 0.5) +
  theme(axis.title.x = element_text(size=10, family = "Arial"), 
        axis.title.y = element_text(size=10, family = "Arial"), 
        axis.text.x = element_text(size=10, colour = 1, family = "Arial"), 
        axis.text.y = element_text(size = 10, colour = 1, family = "Arial")) +
  theme(panel.border = element_rect(fill = NA, size = 1)) 
p2 = DimPlot(object = seu, reduction = "umap", label = FALSE, label.size = 10, pt.size = 0.5, group.by = "orig.ident") +
  theme(axis.title.x = element_text(size=10, family = "Arial"),
        axis.title.y = element_text(size=10, family = "Arial"),
        axis.text.x = element_text(size=10, colour = 1, family = "Arial"),
        axis.text.y = element_text(size = 10, colour = 1, family = "Arial")) +
  theme(panel.border = element_rect(fill = NA, size = 1)) 

legend1 <- cowplot::get_legend(p1)
legend2 <- cowplot::get_legend(p2)
p1 = p1 + theme(legend.position = 'none')
p2 = p2 + theme(legend.position = 'none')
file.name=paste(sample.name, "umap_reso", resol, ".png", sep='')
file.name=paste(dir.name, file.name, sep='/')
save_plot(file = file.name, plot_grid(p1, legend1, p2, legend2, ncol=2, nrow=2), device="png", 
          units="in", dpi = 600, base_width = 10, base_height = 10, limitsize=FALSE)

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“ ポストスクリプトフォントのデータベースにフォントファミリ 'Arial' が見付かりません ”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x

In [22]:
##Extract marker genes

###Input layer
dir.name <- "MarkerGenePlot"

############################Processing layer ###########################
seu.markers = FindAllMarkers(seu, verbose = TRUE, test.use="wilcox", only.pos=TRUE, min.pct=0.1, features.use = NULL, return.thresh=0.05)

#Create heatmap with top10 marker genesdir.create(dir.name)
top10 = seu.markers %>% group_by(cluster) %>% top_n(10, avg_log2FC)
top10 = as.data.frame(top10)
top10  = top10 [!duplicated(top10$gene),]
top10 = top10 %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
top10 = as.data.frame(top10)
file.name=paste(sample.name, "marker_res", resol, ".png", sep='')
file.name <-paste(dir.name, file.name, sep = "/")
p <- DoHeatmap(seu, features = top10$gene, disp.min = -2.5, disp.max = 2.5, size = 8)
ggsave(file = file.name, plot = p, device="png", units="in", dpi = 300,
       width = 20, height = 20, limitsize=FALSE)

seu.markers$cluster = as.numeric(seu.markers$cluster)
seu.markers = seu.markers %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
seu.markers = as.data.frame(seu.markers)
file.name=paste(sample.name, "ALLmarkers_minpct0.1_Adj_p0.05.txt", sep='')
file.name <-paste(dir.name, file.name, sep = "/")
fwrite(seu.markers, file.name, row.names=F, col.names=T, sep="\t", quote=F)
file.name=paste(sample.name, "ALLmarkers_minpct0.1_Adj_p0.05.rda", sep='')
file.name <-paste(dir.name, file.name, sep = "/")
save(seu.markers, file=file.name)

#Output Seurat object in 2nd analysis
file.name=paste(sample.name, "_res", resol, ".rda", sep='')
save(seu, file=file.name)

#Perform integration of scTCR result
#We used the pipeline produced by ImmunoGeneTeqs, Inc. for assembling TCR clone from single-cell TCRseq reads
#and integrating TCR sequence information to Seurat object. (http://immunogeneteqs.com/index.html)

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Warning message in dir.create(dir.name):
“ 'MarkerGenePlot' はすでに存在します ”


### Integration of scTCRseq results into SCT Seurat object

In [3]:
###Define function
#Convert MiXCR output into VDJtools format
Convert <- function(file.name, name.TRA){
    name.input <- str_c(name.TRA, file.name, sep = "/")
    data <- tableread_fast(name.input, header = TRUE, sep = '\t')

    #Extract TCR information from mixcr output
    count.total<-data$cloneCount / data$cloneFraction
    freq<-data$cloneFraction
    cdr3nt<-data$nSeqImputedCDR3
    cdr3aa<-data$aaSeqImputedCDR3
    v<-str_sub(data$bestVHit, end=-4)
    d<-str_sub(data$bestDHit, end=-4)
    j<-str_sub(data$bestJHit, end=-4)
    data3 <- rbind(count.total,freq,cdr3nt,cdr3aa,v,d,j)
    data3 <- as.data.frame(t(data3))
    names(data3) <- c("count.total","freq","cdr3nt","cdr3aa","v","d","j")

    #Extract largest clone in cell barcode and append cell barcode information
    name_out <- str_split(file.name, "_")
    CB <- name_out[[1]][[4]]
    CB_out <- str_split(CB, "\\.")
    CB <- CB_out[[1]][[1]]
    d_out <- data3[1,]
    d_out$CB <- CB

    return(d_out)
}

In [4]:
###Define function
#Create histogram was showing the total number of TCR reads detected in each cell (top left) 
#and the percentage of TCR reads that were the most common among them (bottom left).
#Scatter plot representing these two metrices of cells were also generated
Histogram <- function(d, file.name, dir.output){

  d_count_total <- as.numeric(d$count.total)
  
  #Count histogram
  ppi <- 300
  image.file <- str_c(dir.output, file.name, sep = "/") %>% str_c("histogram.count.tiff", sep = '.') 
  tiff(image.file, width=1.2*ppi, height=0.8*ppi, res=ppi)
  p <- ggplot(NULL, aes(x=d_count_total)) +
    geom_histogram(binwidth=0.1) +
    theme_bw(base_size = 6) +
    labs(x = "Read counts of TCR") +
    theme(
      axis.title.y=element_blank(),
      axis.text.x = element_text(family="Arial"),
      axis.text.y = element_text(family="Arial"),
      axis.title=element_text(size=4)) +
    scale_x_continuous(trans=scales::log2_trans(),
                   breaks=scales::trans_breaks("log2",function(x) 2^x),
                   labels=scales::trans_format("log2",scales::math_format(2^.x)))
  print(p)
  dev.off()

  #Frequency histogram  
  d_freq <- as.numeric(d$freq)
  med_f <- 0.3
  ppi <- 300
  image.file <- str_c(dir.output, file.name, sep = "/") %>% str_c("histogram.freq.tiff", sep = '.')
  tiff(image.file, width=1.2*ppi, height=0.8*ppi, res=ppi)
  p <- ggplot(NULL, aes(x=d_freq)) +
    geom_histogram(binwidth=0.05) +
    theme_bw(base_size = 6) +
    labs(x = "Proportion of the largest TCR read") +
    theme(
      axis.title.y=element_blank(),
      axis.text.x = element_text(family="Arial"),
      axis.text.y = element_text(family="Arial"),
      axis.title=element_text(size=4)) 
  print(p)
  dev.off()

  df <- data.frame(d_count_total, d_freq)
  
  #Scatter plot    
  total <- nrow(subset(df, df$d_freq >= 0))
  lb <- paste(round(100*nrow( subset(df, df$d_freq < 0.6 & df$d_count_total < 2^5 )) / total, digits = 1), "%", sep="" ) 
  rb <- paste(round(100*nrow( subset(df, df$d_freq >= 0.6 & df$d_count_total < 2^5 )) / total, digits = 1), "%", sep="" ) 
  lt <- paste(round(100*nrow( subset(df, df$d_freq < 0.6 & df$d_count_total >= 2^5 )) / total, digits = 1), "%", sep="" ) 
  rt <- paste(round(100*nrow( subset(df, df$d_freq >= 0.6 & df$d_count_total >= 2^5 )) / total, digits = 1), "%", sep="" ) 
  
  ppi <- 300
  image.file <- str_c(dir.output, file.name, sep = "/") %>% str_c("Scatter.tiff", sep = '.')                     
  tiff(image.file, width=1.2*ppi, height=1.2*ppi, res=ppi)
  p <- ggplot(d, aes(x=d_freq, y=d_count_total)) +  
    stat_bin2d(bins=30) +
    scale_fill_gradient(low="lightblue", high="red") +
    theme_bw(base_size = 6) +
    geom_vline(aes(xintercept = 0.6), size=0.25, colour="black") +
    geom_hline(aes(yintercept = 2^5), size=0.25, colour="black") +
    labs(x = "Proportion of the largest TCR read", y = "Read counts of TCR") +
    theme(
      axis.text.x = element_text(family="Arial"),
      axis.text.y = element_text(family="Arial"),
      axis.title=element_text(size=4)) +
    guides(fill=FALSE) +
    scale_y_continuous(trans=scales::log2_trans(),
                   breaks=scales::trans_breaks("log2",function(x) 2^x),
                   labels=scales::trans_format("log2",scales::math_format(2^.x)))
    p <- p + #xlim(0,1) +
    annotate("text", x=-Inf, y=0, hjust=-0.1, vjust=-0.4, label=lb,
             family="Arial",colour="black",size=1.5) +
    annotate("text", x=Inf, y=0, hjust=1.1, vjust=-0.4, label=rb,
             family="Arial",colour="black",size=1.5) +
    annotate("text", x=-Inf, y=Inf, hjust=-0.1, vjust=1.3, label=lt,
             family="Arial",colour="black",size=1.5) +
    annotate("text", x=Inf, y=Inf, hjust=1.1, vjust=1.3, label=rt,
             family="Arial",colour="black",size=1.5) 
  print(p)
  dev.off()  
}

In [5]:
#BD Rhapsody WTA analysis with Seurat package
set.seed(seed = 42)
seurat.name <- "Aoki_3rd_res0.6"
dir.input <- "scTCR_rawdata"
sample.name <- "Ex124TCRBD"
cores <- 12
dir.output <- "scTCR_processing"
#Threshold for valid cell barcode with single TCR sequence
count_th <- 5 #read count threshold for all TCR sequences per cell barcode
freq_th <- 0.6 #proportion threshold for largest TCR
clonotype <- "ABnt" #Definition of clones can be changed between "ABnt", "ABaa", "Bnt", and "Baa"
hashtag <- TRUE #select whether hashtag were used in sample preparation

################################ Processing layer #################################################
dir.create(dir.output)

#Unzip files
files <- list.files(dir.input, ".tar.bz2")
for(file in files){
    name.input <- str_c(dir.input, file, sep = "/")
    bunzip2(name.input, remove=FALSE)
    name.input <- str_remove(name.input, pattern = ".bz2")
    untar(name.input)
    file.remove(name.input)
}

#Convert MiXCR output into VDJtools format
name.TRA <- str_c(sample.name, "_TRAC1_mixcr", sep = "")
files  <- list.files(name.TRA, pattern=".txt")
cl <- makeCluster(cores)
registerDoParallel(cl)
TRA.table <- invisible(foreach(file.name = files,
        .combine = rbind, .packages=c("ggplot2", "extrafont", "stringr", "dplyr", "data.table")) %dopar% {Convert(file.name, name.TRA)})
stopCluster(cl)
unlink(name.TRA, recursive=TRUE)

name.TRB <- str_c(sample.name, "_TRBC1_mixcr", sep = "")
files  <- list.files(name.TRB, pattern=".txt")
cl <- makeCluster(cores)
registerDoParallel(cl)
TRB.table <- invisible(foreach(file.name = files,
        .combine = rbind, .packages=c("ggplot2", "extrafont", "stringr", "dplyr", "data.table")) %dopar% {Convert(file.name, name.TRB)})
stopCluster(cl)
unlink(name.TRB, recursive=TRUE)

#Output histogram and scatter plot for summarizing scTCR status
Histogram(TRA.table, "TRA", dir.output)
Histogram(TRB.table, "TRB", dir.output)

#thresholding
TRA.table$count.total <- as.numeric(TRA.table$count.total)
TRA.table$freq <- as.numeric(TRA.table$freq)
TRB.table$count.total <- as.numeric(TRB.table$count.total)
TRB.table$freq <- as.numeric(TRB.table$freq)
TCRa_th <- subset(TRA.table, count.total >= 2^(count_th) & freq >= freq_th)
TCRb_th <- subset(TRB.table, count.total >= 2^(count_th) & freq >= freq_th)

#Paring TCRa and TCRb by cell barcode
names(TCRa_th) <- c("count.total.A","freq.A","cdr3nt.A","cdr3aa.A","v.A","d.A","j.A", "CB")
names(TCRb_th) <- c("count.total.B","freq.B","cdr3nt.B","cdr3aa.B","v.B","d.B","j.B", "CB")
combined <- merge(TCRa_th, TCRb_th, all=T, by ="CB")

#Generate clone id for further analysis
#Definition of clones can be changed between "ABnt", "ABaa", "Bnt", and "Baa"
if(clonotype=="ABnt"){
    combined <- subset(combined, combined$cdr3nt.A != "UD" & combined$cdr3nt.B != "UD")
    combined$clone.id <- str_c(combined$cdr3nt.A, combined$v.A, combined$j.A, combined$cdr3nt.B, combined$v.B, combined$j.B, sep="_")
}
if(clonotype=="ABaa"){
    combined <- subset(combined, combined$cdr3nt.A != "UD" & combined$cdr3nt.B != "UD")
    combined$clone.id <- str_c(combined$cdr3aa.A, combined$v.A, combined$j.A, combined$cdr3aa.B, combined$v.B, combined$j.B, sep="_")
}
if(clonotype=="Bnt"){
    combined <- subset(combined, combined$cdr3nt.B != "UD")
    combined$clone.id <- str_c(combined$cdr3nt.B, combined$v.B, combined$j.B, sep="_")
}
if(clonotype=="Baa"){
    combined <- subset(combined, combined$cdr3nt.B != "UD")
    combined$clone.id <- str_c(combined$cdr3aa.B, combined$v.B, combined$j.B, sep="_")
}
name.output <- str_c(dir.output, "ABpaird.table", sep = "/") %>% str_c("count_th", count_th, "freq_th", freq_th, clonotype, "csv", sep = ".")
write.csv(combined, name.output, row.names = FALSE)

#load Seurat object
name.input <- str_c(seurat.name, "rda", sep = ".")
load(name.input)

#Extract meta.data and cell BC information
meta.data <- seu@meta.data
meta.data$names <- row.names(meta.data)
CB.info <- str_split(meta.data$names, pattern = "_", simplify = TRUE)
if(hashtag == TRUE){
    meta.data$CB <- CB.info[,3]
} else{
    meta.data$CB <- CB.info[,1]
}

#Merge to Seurat object of SCT data
meta.data <- merge(combined, meta.data, all.y = T, by ="CB")
clone.id.list <- meta.data$clone.id
names(clone.id.list)=as.character(meta.data$names)
seu <- AddMetaData(object = seu, metadata = clone.id.list, col.name = "clone.id")



Warning message:
“Removed 170 rows containing non-finite values (stat_bin).”
Warning message:
“Removed 170 rows containing non-finite values (stat_bin).”
Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Removed 170 rows containing non-finite values (stat_bin2d).”


png 
  2

Warning message:
“Removed 156 rows containing non-finite values (stat_bin).”
Warning message:
“Removed 156 rows containing non-finite values (stat_bin).”
Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Removed 156 rows containing non-finite values (stat_bin2d).”


png 
  2

### SCT analysis after integrating scTCRseq results

In [6]:
# Introduce Gene Scores using AddModuleScore function

#Put signature gene set into "Signatures" directory
dir.input <- "Signatures"

########################## Processing layer #############################
#load data
files  <- list.files(dir.input, pattern="Sig.")

for(i in files){
  i <- str_c(dir.input, i, sep = "/")
  gene_list <- read.table(i, header = TRUE)
  gene_list <- list(as.vector(gene_list$Genes))
  sig_name <- str_remove(i, dir.input)
  sig_name <- str_remove(sig_name, "/Sig.")
  sig_name <- str_remove(sig_name, ".txt")
  seu <- AddModuleScore(object = seu, 
                        ctrl=10,
                        features = gene_list,
                        name=sig_name)
}

#Output Seurat object
name.output <- str_c(seurat.name, "scTCRmerged", clonotype, "AddSig", "rda", sep = ".")
save(seu, file=name.output)

Warning message:
“The following features are not present in the object: Ifna1, not searching for symbol synonyms”
Warning message:
“The following features are not present in the object: Cd244, not searching for symbol synonyms”


In [7]:
### Calculate frequency and cluster distribution of each clones detected by scTCR

###Input layer
dir.name <- "scTCR_analysis"

########################## Processing layer #############################
dir.create(dir.name)

#load and output metadata
meta.data <- as.data.frame(seu@meta.data)
name.out <- str_c(dir.name, "meta.data.csv", sep = "/")
write.csv(meta.data, name.out, row.names = TRUE)

#Summarize the count and freqnecy of clones
tmp = meta.data %>% group_by(`clone.id`) %>%
dplyr::summarise(count = n()) %>%
dplyr::arrange(desc(count))
tmp = as.data.frame(tmp)
tmp <- tmp[!is.na(tmp$clone.id), ] #Delete TCR unidentified cells
tmp[,3]=tmp[,2]/sum(tmp[,2])
colnames(tmp)=c("ntSeq_TRA_TRB_freq", "CloneCount", "CloneFreq")
tmp <- tmp[order(tmp$ntSeq_TRA_TRB_freq),]
temp_count <- as.vector(tmp$CloneCount)
temp_freq <- as.vector(tmp$CloneFreq)

#Summarize the distribution of clones to clusters defined by SCT
tmp_out = table(meta.data$seurat_clusters, meta.data$clone.id)
tmp_out2 <- as.data.frame(tmp_out, row.names = NULL,
              responseName = "Freq", stringsAsFactors = TRUE,
              sep = "", base = list(LETTERS))
names(tmp_out2) <- c("Clust", "names", "Freq")
tmp_out2 <- dcast(tmp_out2, names ~ Clust)

#Integrate count/frequency information and cluster distribution
tmp_out3 <- cbind(temp_count, temp_freq, tmp_out2)

#Rank the clones
tmp_out3 <- tmp_out3[order(tmp_out3$temp_freq, decreasing=T),]
rank <-  1:nrow(tmp_out3)
tmp_out3 <- cbind(rank, tmp_out3)
tmp_out3 <- data.frame(tmp_out3)
tmp_out3$rank <- paste("Top",tmp_out3$rank, sep="")

#Output
name.out <- str_c("clone_within_cluster.txt", sep = ".")
name.out <- paste(dir.name, name.out, sep='/')
write.table(tmp_out3, name.out, row.names=F, col.names=T, sep="\t", quote=F) 

Using Freq as value column: use value.var to override.



In [8]:
### Detect dLN-Tumor OL clones

###Input layer
#Specify query files: frequency and cluster distribution table of scTCR clones
dir.query <- "scTCR_analysis"
name.query <- "clone_within_cluster"

#Specify the repertoire data of dLN
dir.input <- "BulkTCR_rawdata"
ID.input <- "BDscTCR_dLN" 

#Specify the condition for detecting overlappping clones
extract <- c("cdr3nt", "v", "j")

#Output directory
dir.output <- "scTCR_analysis"

cores <- 2

########################## Processing layer #############################
#load dLN repertoire files
files  <- list.files(dir.input, pattern=ID.input)

#load query files
query.input <- str_c(name.query, "txt", sep = ".")
query.input <- str_c(dir.query, query.input, sep = "/")
clone.q <- read.table(query.input, header = TRUE)

#Reconstruct the information of clones for searching overlap
clone.name <- as.data.frame(str_split(clone.q$names, pattern = "_", simplify = TRUE))
names(clone.name) <- c("Ant", "Av", "Aj", "Bnt", "Bv", "Bj")
clone.q$TCRb <- str_c(clone.name$Bnt, clone.name$Bv, clone.name$Bj)

#====================================================================
Basic <- function(dir.input, file.name, clones.q, extract, name.query){ 
  #load dLN reprtoire files
  name.input <- str_c(dir.input, file.name, sep = "/")
  d <- tableread_fast(name.input, header = TRUE, sep="\t", quote = "\"")
  
  #Reconstruct the information of clones for searching overlap
  d <- dplyr::select(d, extract)
  d$TCRb <- str_c(d$cdr3nt, d$v, d$j)
  
  #Search clones overlapped with Tumor scTCR repertoire
  d_output <- merge(clone.q, d, by = "TCRb", all.x = T)
    
  return(d_output)
}

#===============================================================

t<-proc.time()
cl <- makeCluster(cores)
registerDoParallel(cl)   
out <- foreach(file.name = files,
        .packages=c("data.table", "stringr", "dplyr", "ggplot2"),
        .combine = rbind) %dopar% {Basic(dir.input, file.name, clones.q, extract)}
stopCluster(cl)
proc.time()-t


   user  system elapsed 
  0.028   0.002   1.932 

In [9]:
### Integrate dLN-tumor OL status (oligoclona OL / polyclonal OL / nonOL) and gene scores

###Define Oligoclonal / Polyclonal fraction of dLN-tumor OL
out$OL <- "nonOL"
#Extract OL clones
data_LT <- dplyr::filter(out, !is.na(cdr3nt))
data_nonOL <- dplyr::filter(out, is.na(cdr3nt))
#Determine Oligoclonal / Polyclonal fraction
data_LT <- data_LT[order(data_LT$temp_freq, decreasing = T),]
count.poly <- nrow(data_LT)-10
data_LT$OL <- c(rep("Oligo", times = 10), rep("Poly", times = count.poly))
out <- rbind(data_LT, data_nonOL)

###Summarize Gene score for each TCR clone
meta.data <- as.data.frame(seu@meta.data)
meta.data.calc <- dplyr::select(meta.data, c("clone.id", "Cytotoxicity1", "Proliferation1", "Tumor.Prog1", "Tumor.Term1"))
out_score <- aggregate(x=meta.data.calc[c("Cytotoxicity1", "Proliferation1", "Tumor.Prog1", "Tumor.Term1")],
                 by=list(meta.data.calc$clone.id), FUN=mean)

#Integration
out <- merge(out, out_score, by.x = "names", by.y = "Group.1", all.x = T)

#Output
dir.create(dir.output)
name.out <- str_c(name.query, "dLNOL.csv", sep = ".")
name.out <- str_c(dir.output, name.out, sep = "/")

write.csv(out, name.out, row.names = FALSE)

Warning message in dir.create(dir.output):
“'scTCR_analysis' already exists”
